- **u.data**: Chứa toàn bộ các đánh giá của 943 người dùng cho 1682 bộ phim. Mỗi người dùng đánh giá ít nhất 20 movie. Thông tin về thời điểm đánh giá cũng được cho nhưng chúng ta không sử dụng trong ví dụ này.
- **a.base**, **ua.test**, **ub.base**, **ub.test**: là hai cách chia toàn bộ dữ liệu ra thành hai tập con, một cho huấn luyện, một cho kiểm thử. Chúng ta sẽ thực hành trên ua.base và ua.test.
- **u.user**: Chứa thông tin về người dùng, bao gồm: id, tuổi, giới tính, nghề nghiệp, zipcode (vùng miền), vì những thông tin này cũng có thể ảnh hưởng tới sở thích của các người dùng. Tuy nhiên, trong ví dụ này, chúng ta sẽ không sử dụng các thông tin này, trừ thông tin về id để xác định các user khác nhau.
-  **u.genre**: Chứa tên của 19 thể loại phim. Các thể loại bao gồm: unknown, Action, Adventure, Animation, Children‘s, Comedy, Crime, Documentary, Drama, Fantasy, Film−Noir, Horror, Musical, Mystery, Romance, Sci−Fi, Thriller, War, Western,
- **u.item**: thông tin về mỗi bộ phim. Ví dụ:

  1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
  |0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0  
  2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
  |0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0  
  3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)  
  |0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0  
  4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)  
  |0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0

- Trong mỗi dòng, chúng ta sẽ thấy id của phim, tên phim, ngày phát hành, link trên imdb, và các số nhị phân 0, 1 phía cuối để chỉ ra bộ phim thuộc các thể loại nào trong 19 thể loại đã cho trong u.genre. Một bộ phim có thể thuộc nhiều thể loại khác nhau. Thông tin về thể loại này sẽ được dùng để xây dựng item profiles.

In [26]:
from __future__ import print_function
import numpy as np
import pandas as pd

# Reading user file:
u_cols = ["user_id", "age", "sex", "occupation", "zip_code"]
users = pd.read_csv("ml-100k/u.user", sep="|", names=u_cols)
n_users = users.shape[0] # (axis 0): Trả về kích thước của mảng trong chiều đầu tiên (row) -> số hàng
print("Number of users:", n_users)

#Reading ratings file:
r_cols = ["user_id", "movie_id", "rating", "unix_timestamp"]
ratings_base = pd.read_csv("ml-100k/ua.base", sep="\t", names=r_cols)
ratings_test = pd.read_csv("ml-100k/ua.test", sep="\t", names=r_cols)

rate_train = ratings_base.values
rate_test = ratings_test.values

print("Number of traing rates:", rate_train.shape[0])
print("Number of test rates:", rate_test.shape[0])
print("---------------------")
print(users.head());
print("---------------------")
print(ratings_base.head());
print("---------------------")
print(rate_train)

Number of users: 943
Number of traing rates: 90570
Number of test rates: 9430
---------------------
   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213
---------------------
   user_id  movie_id  rating  unix_timestamp
0        1         1       5       874965758
1        1         2       3       876893171
2        1         3       4       878542960
3        1         4       3       876893119
4        1         5       3       889751712
---------------------
[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


In [30]:
#Reading items file:
i_cols = ["movie id", "movie title" ,"release date","video release date", "IMDb URL",
          "unknown", "Action", "Adventure", "Animation", "Children\"s", "Comedy", "Crime",
          "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

items = pd.read_csv("ml-100k/u.item", sep="|", names=i_cols, encoding='latin-1')
n_items = items.shape[0]
print("Number of items:", n_items)
print("---------------------")
print(items.head());

Number of items: 1682
---------------------
   movie id        movie title release date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Children"s  ...  Fantasy  Film-Noir  Horror  Musical  \
0          0          1          

In [8]:
# Vì ta đang dựa trên thể loại của phim để xây dựng profile, ta sẽ chỉ quan tâm tới 19 giá trị nhị phân ở cuối mỗi hàng.
X0 = items.values
X_train_counts = X0[:, -19:]


In [10]:
# Tiếp theo, chúng ta hiển thị một số hàng đầu tiên của ma trận rate_train
print(rate_train[:4, :])
# Hàng thứ nhất được hiểu là user thứ nhất đánh giá movie thứ nhất 5 sao. Cột cuối cùng là một số chỉ thời điểm đánh giá, chúng ta sẽ bỏ qua thông số này.

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]]


In [31]:
# Tiếp theo, chúng ta sẽ xây dựng feature vector cho mỗi item dựa trên ma trận thể loại phim và feature TF-IDF

#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ="l2")
X = transformer.fit_transform(X_train_counts.tolist()).toarray()

print(X)
# Sau bước này, mỗi hàng của X tương ứng với vector đặc trưng của một bộ phim.

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.54 0.65 ... 0.54 0.   0.  ]
 [0.   0.   0.   ... 1.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [12]:
# Với mỗi người dùng, chúng ta cần đi tìm những bộ phim nào mà người dùng đó đã đánh giá, và giá trị của các rating đó.
def get_items_rated_by_user(rate_matrix, user_id):
  """
  return (item_ids, scores)
  """
  y = rate_matrix[:,0] # all users
  # item indices rated by user_id
  # we need to +1 to user_id since in the rate_matrix, id starts from 1
  # but id in python starts from 0
  ids = np.where(y == user_id +1)[0] # `ids` sẽ là một mảng chứa các chỉ mục của các phần tử trong mảng `y` có giá trị bằng `user_id + 1`
  item_ids = rate_matrix[ids, 1] - 1 # index starts from 0
  scores = rate_matrix[ids, 2]
  # Trong rate matrix, cột đầu tiên là user, cột thứ 2 là movie, cột thứ 3 là rating

  return (item_ids, scores)


In [13]:
# Bây giờ, ta có thể đi tìm các hệ số của Ridge Regression (W, b) cho mỗi người dùng
from sklearn.linear_model import Ridge
from sklearn import linear_model
d = X.shape[1] # data dimension (columns) -> số cột
W = np.zeros((d, n_users))
b = np.zeros(n_users)
for n in range(n_users):
  ids, scores = get_items_rated_by_user(rate_train, n)
  model = Ridge(alpha=0.01, fit_intercept = True)
  Xhat = X[ids, :]
  model.fit(Xhat, scores)
  W[:, n] = model.coef_ # weights
  b[n] = model.intercept_ # biases

In [32]:
# Sau khi tính được các hệ số W và b, rating mà mỗi người dùng đánh giá mỗi bộ phim được dự đoán bằng cách:

# predicted scores
Yhat = X.dot(W) + b
print(Yhat)

[[2.92 4.   1.9  ... 5.04 4.78 3.17]
 [2.8  3.44 3.3  ... 1.1  4.76 3.7 ]
 [3.48 1.56 1.27 ... 7.36 4.19 3.81]
 ...
 [4.13 4.1  3.02 ... 9.54 4.64 3.27]
 [3.57 3.45 2.93 ... 3.68 3.66 2.76]
 [4.13 3.8  3.1  ... 9.54 4.42 3.93]]


In [15]:
# Dưới đây là một ví dụ với người dùng có id là 10.
n = 10
np.set_printoptions(precision=2) # 2 digits after .
ids, scores = get_items_rated_by_user(rate_test, n)
print("Rated movies ids :", ids )
print("True ratings :", scores)
print("Predicted ratings:", Yhat[ids, n])


Rated movies ids : [ 37 109 110 226 424 557 722 724 731 739]
True ratings : [3 3 4 3 4 3 5 3 3 4]
Predicted ratings: [3.18 3.13 3.42 3.09 3.35 5.2  4.01 3.35 3.42 3.72]


In [17]:
# Để đánh giá mô hình tìm được, chúng ta sẽ sử dụng Root Mean Squared Error (RMSE), tức căn bậc hai của trung bình cộng bình phương của lỗi
def evaluate(Yhat, rates, W, b):
  se = cnt = 0
  for n in range(n_users):
    ids, scores_truth = get_items_rated_by_user(rates, n)
    scores_pred = Yhat[ids, n]
    e = scores_truth - scores_pred
    se += (e*e).sum(axis = 0)
    cnt += e.size
  return np.sqrt(se/cnt)
print("RMSE for training: %.2f" %evaluate(Yhat, rate_train, W, b))
print("RMSE for test : %.2f" %evaluate(Yhat, rate_test, W, b))


RMSE for training: 0.91
RMSE for test : 1.27


Như vậy, với training set, sai số vào khoảng 0.91 (sao); với test set, sai số lớn hơn một chút, khoảng 1.27. Các kết quả này chưa thực sự tốt vì mô hình đã được đơn giản hoá quá nhiều. Kết quả tốt hơn có thể được thấy trong các chương tiếp theo về collaborative filtering.
